<a href="https://colab.research.google.com/github/KiranH1007/-MLB-fan-engagement-project/blob/build-fan-engagement-application-using-ai-models/Live%20Game%20Sentiment%20Analysis/Live%20Game%20Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authenticate Google Cloud in Colab
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Install Required Libraries
!pip install google-cloud-storage google-cloud-bigquery nltk pandas

In [ ]:
from google.cloud import storage

# Initialize GCS client
storage_client = storage.Client()

# Specify your GCS bucket and file name
bucket_name = 'gcp-mlb-hackathon-2025' # Make sure this is the correct bucket name
blob_name = 'datasets/mlb-fan-content-interaction-data/2025-mlb-fan-favs-follows.json'

# Get the blob directly without getting the bucket metadata first
# This assumes the bucket is public and blob_name includes the full path
blob = storage_client.bucket(bucket_name).blob(blob_name)

# Download the file
raw_data = blob.download_as_text()

# Print the data (optional)
#print(raw_data[0:100])

In [ ]:
# load json with pandas
# Data set is 2025-mlb-fan-favs-follows.json
import pandas as pd
import json
from google.cloud import storage

# Initialize GCS client
storage_client = storage.Client()

# Specify your GCS bucket and file name
bucket_name = 'gcp-mlb-hackathon-2025'
blob_name = 'datasets/mlb-fan-content-interaction-data/2025-mlb-fan-favs-follows.json'

# Get the blob directly
blob = storage_client.bucket(bucket_name).blob(blob_name)

# Download the file as bytes
data = blob.download_as_bytes()

# Decode the bytes to a string
raw_data = data.decode('utf-8')

# Load the data using json.loads to handle possible JSON Lines format
try:
    # Attempt to load as a single JSON object
    json_data = json.loads(raw_data)
    df_fanfav = pd.DataFrame([json_data]) # Enclose in a list if it's a single object
except json.JSONDecodeError:
    # If single object fails, try loading as JSON Lines
    json_data = [json.loads(line) for line in raw_data.splitlines() if line.strip()]
    df_fanfav = pd.DataFrame(json_data)

# Preview data
#print(df_fanfav.head())

print(df_fanfav.info())

In [ ]:
# load json with pandas
# data set is mlb-fan-content-interaction-data-000000000000.json

'''  sample code for analysis of one dataset
import pandas as pd
import json
from google.cloud import storage

# Initialize GCS client
storage_client = storage.Client()

# Specify your GCS bucket and file name
bucket_name = 'gcp-mlb-hackathon-2025'
blob_name = 'datasets/mlb-fan-content-interaction-data/mlb-fan-content-interaction-data-000000000000.json'

# Get the blob directly
blob = storage_client.bucket(bucket_name).blob(blob_name)

# Download the file as bytes
data = blob.download_as_bytes()

# Decode the bytes to a string
raw_data = data.decode('utf-8')

# Load the data using json.loads to handle possible JSON Lines format
try:
    # Attempt to load as a single JSON object
    json_data = json.loads(raw_data)
    df3 = pd.DataFrame([json_data]) # Enclose in a list if it's a single object
except json.JSONDecodeError:
    # If single object fails, try loading as JSON Lines
    json_data = [json.loads(line) for line in raw_data.splitlines() if line.strip()]
    df3 = pd.DataFrame(json_data)

# make single column for all content to content id due to similar data like slug and content_headline
#
# Redundancy: Storing essentially the same information twice.
# Increased storage: Larger data size due to duplicated information.
# Potential inconsistency: If one field is updated and the other isn't, it could create discrepancies.
#
#step 1 observation
# print(len(df3['slug']))
# print(len(df3['content_headline']))
# print(df3['slug'][100])
# print(df3['content_headline'][100])
# print(df3['content_type'][100])

# step 2 combining
# Prioritize One Field:
# If slug and content_headline convey nearly identical information, choose the one that's more concise and
# consistently formatted (likely slug) as the primary identifier.
# In most cases, the slug is a better choice for the primary identifier because:
# Conciseness: Slugs are typically shorter and more URL-friendly than content headlines.
# Consistency: Slugs are often generated programmatically and have a more consistent format compared to headlines, which might contain variations in punctuation and capitalization.

# remove extra column
df3.drop(columns=['content_headline'], inplace=True)
df3['content_id'] = df3['content_type'] + ':' + df3['slug'].str.lower()
df3.drop(columns=['slug'], inplace=True)
df3.drop(columns=['content_type'], inplace=True)

#print(df3['content_id'][100])


# similarly for date time utc as date_time_date is same as utc
df3.drop(columns=['date_time_date'], inplace=True) # Drop the redundant column

# Preview data
df3.head()

#print(df3.info())

'''

In [ ]:
# load json with pandas
# data set altogether with  multiple json files
# Note ******** currently doing with 5 files later do the Scalable
import pandas as pd
import json
from google.cloud import storage
import re

# Initialize GCS client
storage_client = storage.Client()

# Specify your GCS bucket and file name
bucket_name = 'gcp-mlb-hackathon-2025'
file_pattern = 'datasets/mlb-fan-content-interaction-data/'

# List to store dataframes from each file
all_dfs = []

blobs = storage_client.list_blobs(bucket_name, prefix=file_pattern)
# for blob in blobs:
#     print(blob.name)

filtered_blobs = [blob for blob in blobs if re.match(r'datasets/mlb-fan-content-interaction-data/mlb-fan-content-interaction-data-\d+\.json$', blob.name)]

# limiting the dataset
filtered_blobs = filtered_blobs[:5]

for blob_name in filtered_blobs:

       print(f"Processing file: {blob_name.name}")
       # Get the blob directly
       blob = storage_client.bucket(bucket_name).blob(blob_name.name)

       # Download the file as bytes
       data = blob.download_as_bytes()

       # Decode the bytes to a string
       raw_data = data.decode('utf-8')

       # Load the data using json.loads
       try:
           json_data = json.loads(raw_data)
           df = pd.DataFrame([json_data])
       except json.JSONDecodeError:
           json_data = [json.loads(line) for line in raw_data.splitlines() if line.strip()]
           df = pd.DataFrame(json_data)

       #print(df.head())
       # Data Cleaning: Remove Redundancy (similar to your example)
       df.drop(columns=['content_headline'], inplace=True, errors='ignore')  # Ignore if column doesn't exist
       df['content_id'] = df['content_type'] + ':' + df['slug'].str.lower()
       df.drop(columns=['slug', 'content_type'], inplace=True, errors='ignore')
       df.drop(columns=['date_time_date'], inplace=True, errors='ignore')

       # Add the cleaned dataframe to the list
       all_dfs.append(df)

final_df = pd.concat(all_dfs, ignore_index=True)

final_df.head()

In [ ]:
# Analysis of DataFrames are named df, df2, df3
common_columns = list(set(df_fanfav.columns) & set(final_df.columns))
print("Common Columns:", common_columns)


In [ ]:
#without cleanup and duplicate columns

import pandas as pd

# it should be outer for all unique data and inner for common data

# Merge the result with the third dataset which has content type headline
merged_df = pd.merge(df_fanfav, final_df, on=common_columns, how='outer')

print(merged_df.count())

merged_df.head()